In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.cuda.amp import autocast, GradScaler
import torch
import time

tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained("distilgpt2").to("cuda")
model.train()

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
scaler = GradScaler()

input_texts = ["AMP test sentence."] * 4
inputs = tokenizer(input_texts, return_tensors="pt", padding=True, truncation=True).to("cuda")

start_time = time.time()
with autocast():
    outputs = model(**inputs, labels=inputs["input_ids"])
    loss = outputs.loss

scaler.scale(loss).backward()
scaler.step(optimizer)
scaler.update()
elapsed = time.time() - start_time

print(f"AMP Loss: {loss.item():.4f} | Time: {elapsed:.4f}s")
print(f"Max GPU Memory (MB): {torch.cuda.max_memory_allocated() / 1e6:.2f}")


<ipython-input-8-1bb37337589e>:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
<ipython-input-8-1bb37337589e>:18: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


AMP Loss: 8.2111 | Time: 0.3664s
Max GPU Memory (MB): 5513.04
